# PalData (new)

Data can be retrieved or loaded into the SwarmPAL framework using the `PalData` construct. This construct can hold multiple datasets (in the form of `PalDataItem`'s), connecting in one direction to tools to fetch data, and in the other direction to tools that process and visualise data.

In [ ]:
from swarmpal.io import PalData, PalDataItem


A `PalData` object is created by passing a series of `PalDataItem` objects. Each `PalDataItem` can be created through different methods...

## PalDataItem from VirES

In [ ]:
# Pass parameters to the .from_vires method to fetch data from VirES (via viresclient)
item_v = PalDataItem.from_vires(
    server_url="https://vires.services/ows",
    collection="SW_OPER_MAGA_LR_1B",
    measurements=["F", "B_NEC"],
    models=["IGRF"],
    start_time="2016-01-01T00:00:00",
    end_time="2016-01-01T00:00:10",
    kwargs=dict(asynchronous=False, show_progress=False),
)

The above code will just create the object (with some light validation), without fetching and loading the data. To do that, run `.initialise()`, and then the dataset will be made available in the `.xarray` attribute:

In [ ]:
item_v.initialise()
item_v.xarray

## PalDataItem from HAPI

In [ ]:
# Pass parameters to the .from_hapi method to fetch data from any HAPI servier (via hapiclient)
# Note that the available parameters are different from VirES
item_h = PalDataItem.from_hapi(
    server_url="https://vires.services/hapi",
    collection="SW_OPER_MAGA_LR_1B",
    measurements=["F", "B_NEC"],
    start_time="2016-01-01T00:00:00",
    end_time="2016-01-01T00:00:10",
    options=dict(logging=False),
)
item_h.initialise()
item_h.xarray

## PalDataItem from local file or object

In [ ]:
# Get a copy of the data above to use as a test
ds = item_h.xarray.copy()
ds.to_netcdf("test.nc")
# Create identical Pal objects from that data
item_f = PalDataItem.from_file("test.nc")
item_m = PalDataItem.from_manual(ds)

## Assembling PalData objects

Any number of items can be attached to `PalData` (and the `.initialise()` is passed through to each item).

Items can either be referenced by a simple index:

In [ ]:
my_pal = PalData(item_v, item_h, item_f, item_m)
my_pal.initialise()
my_pal[0], my_pal[1], my_pal[2], my_pal[3]

... or by passing keywords:

In [ ]:
my_pal = PalData(one=item_v, two=item_h, three=item_f, four=item_m)
my_pal.initialise()
my_pal["one"], my_pal["two"], my_pal["three"], my_pal["four"]

Putting the above steps together, we can create a `PalData` directly with:

In [ ]:
my_pal = PalData(
    one=PalDataItem.from_file("test.nc"),
    two=PalDataItem.from_file("test.nc"),
)

In [ ]:
params = dict(
    server_url="https://vires.services/ows",
    collection="SW_OPER_MAGA_LR_1B",
    measurements=["F", "B_NEC"],
    models=["IGRF"],
    start_time="2016-01-01T00:00:00",
    end_time="2016-01-01T00:00:10",
    kwargs=dict(asynchronous=False, show_progress=False),
)
my_pal = PalData(PalDataItem.from_vires(**params))
my_pal.initialise()
my_pal[0].xarray

In [ ]:
# A final cleanup to remove that file
from os import remove

remove("test.nc")